## Hello Braket!

Wondering how to access Aquila? Say hello to Braket! 

The AWS Braket interface allows you to send tasks from your laptop to quantum processors via the AWS cloud infrastructure. This tutorial will give you a first taste of what this workflow looks like. It's simple: Define the arrangement of atoms you want to use and specify how you want to manipulate those atoms with our lasers. Submit the program and analyse the results. 

Ready to explore what's possible with Aquila? Let's dive straight in!


### Goal

To say hello to the world of neutral atoms, let's make use of the [Rydberg blockade](https://queracomputing.github.io/Bloqade.jl/dev/tutorials/1.blockade/main/) - the phenomenon at the heart of our quantum computing architecture. Here, we will use it to entangle two atoms.

### Register

To achieve our goal, we first need to define our quantum computer's register, consisting of those two atoms separated by 5.5μm. 

In [ ]:
from braket.ahs.atom_arrangement import AtomArrangement
import numpy as np

a = 5.5e-6  # meters

register = AtomArrangement()
register.add([0, 0])
register.add([a, 0.0]);

This two body scenario is the simplest case you'll encounter as you work your way towards the complex [many-body physics simulations](https://github.com/aws/amazon-braket-examples/blob/ahs/examples/analog_hamiltonian_simulation/02_Ordered_phases_in_Rydberg_systems.ipynb) made possible by QuEra. But let's take it easy and take a first look at our simple register:

In [ ]:
from utils import show_register

show_register(register)

### Hamiltonian

The next ingredient we need to specify is the Hamiltonian. It's the energy function that governs the behaviour of our atoms, including their interactions. In case this sounds unfamiliar, feel free to read up on the physics background on our open source platform [Bloqade](https://queracomputing.github.io/Bloqade.jl/dev/tutorials/1.blockade/main/).

In [ ]:
from braket.ahs.hamiltonian import Hamiltonian

H = Hamiltonian()

In the lab, the Hamiltonian is implemented by applying lasers to the atoms. Since we're keeping it simple, we only want to tell the system to activate one set of lasers called the Rabi drive. When these lasers are applied to our atoms, they are put into superposition and begin to oscillate between their ground state |g> and excited state |r>. 

For our example, we will choose a constant laser frequency equal to $\Omega=2.5\times10^6$ rad/s together with a time duration of $t = \pi/\sqrt{2}$ for this pulse. The other laser parameters available in our lab, called detuning and phase in the code below, will simply be set to zero. The rest of the code simply ensures that $\Omega$ is turned on and off in a way compatible with the experimental setup.

In [ ]:
from braket.ahs.time_series import TimeSeries
from braket.ahs.driving_field import DrivingField
from utils import rabi_pulse, zero_time_series_like

omega_const = 2.5e6  # rad / seconds
rabi_phase = np.pi/np.sqrt(2) # rad    
omega_slew_rate_max = float(rydberg.rydbergGlobal.rabiFrequencySlewRateMax) # rad/s^2       

time_points, amplitude_values = rabi_pulse(rabi_phase, omega_const, omega_slew_rate_max)

amplitude = TimeSeries()
for t, v in zip(time_points, amplitude_values):
    amplitude.put(t, v)

detuning = zero_time_series_like(amplitude) 
phase = zero_time_series_like(amplitude) 

H += DrivingField(amplitude=amplitude, detuning=detuning, phase=phase)

However, that's not the full story. The Rabi drive term alone doesn't tell us how the atoms interact. But we wanted to entangle the two atoms, didn´t we? That's where the Rydberg blockade comes in. The key idea is that within a certain distance - the so-called blockade radius - only one atom will be excited into an |r> state. Having more excitations will cost too much energy. This means that the states of our two neighboring atoms will depend on each other: If one atom is in |r>, the other should be in |g> and vice versa.

In the current version of the AHS module in Braket, this interaction term is automatically calculated from the atom positions. Hence, we don't need to worry about specifying it explicitly. Our Hamiltonian is all set!

### Task

Now, we can combine the register and Hamiltonian into a program. In particular, this program falls within the class of Analog Hamiltonian Simulation (AHS). If you're curious about other types of quantum computing, take a look at [gate-based circuits](https://github.com/aws/amazon-braket-examples/blob/main/examples/getting_started/1_Running_quantum_circuits_on_simulators/1_Running_quantum_circuits_on_simulators.ipynb) or tutorials on [quantum annealing](https://github.com/aws/amazon-braket-examples/blob/main/examples/quantum_annealing/Dwave_TravelingSalesmanProblem/Dwave_TravelingSalesmanProblem.ipynb).

In [ ]:
from braket.ahs.analog_hamiltonian_simulation import AnalogHamiltonianSimulation

ahs_program = AnalogHamiltonianSimulation(
    register=register, 
    hamiltonian=H
)

To make the program compatible with the quantum hardware, we still need to slice it into discrete time steps:

In [ ]:
discretized_ahs_program = ahs_program.discretize(device)

And now for the truly exciting part! Let's bring Aquila into the game by specifying it as the device we want to send our program to. 

In [ ]:
from braket.aws import AwsDevice 

device = AwsDevice("arn:aws:braket:us-east-1::device/qpu/quera/Aquila")

nshots = 1000
#(Comment on shots can be excluded, depending on audience:)
# When you run the program, don't forget to specify the number of shots. This will instruct the machine to run the program many times. 
# Why is this important? Well, quantum mechanics is probabilistic meaning that each time you run the same experiment, you might get a different answer. 
# Hence, we want to later study the averages of multiple runs.)
task = device.run(discretized_ahs_program, shots=n_shots)

<div class="alert alert-block alert-info">
    <b>Note: </b> Running this program on the Aquila processor will incur a cost of 0.30 USD for submitting the program and 0.0X USD per shot. To run this example as is, will cost you Y USD.
</div>

The results (once the task is completed) can be downloaded directly into an object in the python session.

In [ ]:
result = task.result()

### Analyzing the result from Aquila

Finally, let's have a look at our very first experiment. To confirm that we indeed arrive at a maximally entangled state, we first collect the measurement results, followed by counting the number of occurence of $|gr\rangle$ and $|rg\rangle$ respectively.

In [ ]:
def get_counters_from_result(result):
    post_sequences = [list(measurement.post_sequence) for measurement in result.measurements]
    post_sequences = ["".join(['r' if site==0 else 'g' for site in post_sequence]) for post_sequence in post_sequences]

    counters = {}
    for post_sequence in post_sequences:
        if post_sequence in counters:
            counters[post_sequence] += 1
        else:
            counters[post_sequence] = 1
    return counters

get_counters_from_result(result)

We see that it is indeed very unlikely for the 1st and 2nd atoms to be excited to the Rydberg states simultaneously due to the Rydberg blockade effect. In other words, the state of the 1st and 2nd atom depend on each other - they are entangled.

### Plotting the result

A picture says more than a thousand words - so let's plot our result! 
Before we plot, however, let's add more time steps to our experiment so that we can visualize the time evolution of our two atom system.

In [ ]:
N_cycles = 2
N_steps = N_cycles * 10  # We take 10 measurements per Rabi cycle

rabi_phase_step = np.pi/(np.sqrt(2)*10) # rad    /  Aim: take X = 10 measurements per Rabi cycle

def get_drive(i):
    time_points, amplitude_values = rabi_pulse(rabi_phase_step * i, omega_const, omega_slew_rate_max)

    amplitude = TimeSeries()
    for t, v in zip(time_points, amplitude_values):
        amplitude.put(t, v)

    detuning = zero_time_series_like(amplitude) 
    phase = zero_time_series_like(amplitude) 

    return DrivingField(amplitude=amplitude, detuning=detuning, phase=phase)

drives = [get_drive(i) for i in range(N_steps)]     # check inline notation here

In [ ]:
# Check code below: Idea is to encode |gg> = 0 + 0 = 0 and |gr> = |rg> = 0 + 1 = 1 + 0 = 1. The plotted curve should then hopefully oscillate between 0 and 1.
# In current version, 1000 shots are taken 10 times per Rabi cycle. Can be modified as needed. 

T = N_steps * rabi_pulse_step / omega_const
time = np.arange(0, T, rabi_pulse_step / omega_const)
filling = zeros(N_steps)

for k, drive in enumerate(drives):
    ahs_program = AnalogHamiltonianSimulation(register=register, hamiltonian=drive)
    discretized_ahs_program = ahs_program.discretize(device)
    task = device.run(discretized_ahs_program, shots=n_shots)
    result = task.result()

    counter = 0
    for measurement in result.measurements:
        counter += sum(measurement.post_sequence)                 # I'm assuming the post_sequences are stored as arrays?
    
    filling[i] = (counter/nshots) 


plt.plot(time, filling)
# To-do: plotting details

To-do: Comment on plot & write conclusion.

Send user over to detailed Braket documentation.

Still not convinced that you've created an entangled pair of atoms? You've got a good nose, fellow quantum explorer! Indeed, the most definitive way to prove entanglement is to change the relative phase between the two-atom states |rg> and |gr> and perform proper state tomography. This, however, requires local rotations applied to individual atoms, i.e. we need our to be able to address each atom separately. Guess what: Our engineers are working on improving laser control to achieve just that. So stay tuned for local tunability! 

Until then, enjoy exploring [Braket](https://docs.aws.amazon.com/braket/latest/developerguide/braket-devices.html). Aquila is at your fingertips!